In [25]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
# import polars as pl
import pandas as pd

In [26]:
conf = SparkConf().set('spark.ui.port', '4045')\
  .set("google.cloud.auth.service.account.enable", "true")\
  .set("google.cloud.auth.service.account.json.keyfile", "/opt/spark/credentials/google-credential.json")
spark = SparkSession.builder.appName("test").config(conf = conf).master("local[*]").getOrCreate()
# spark.conf.set("google.cloud.auth.service.account.enable", "true") 
# spark.conf.set("google.cloud.auth.service.account.json.keyfile", "/opt/spark/credentials/google-credential.json")

In [27]:
def get_spark_schema():
  from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, FloatType, DoubleType, StringType
  return StructType([
    StructField("MMSI", StringType(), False),
    StructField("BaseDateTime", TimestampType(), False),
    StructField("LAT", DoubleType(), False),
    StructField("LON", DoubleType(), False),
    StructField("SOG", FloatType(), False),
    StructField("COG", FloatType(), False),
    StructField("Heading", FloatType(), True),
    StructField("VesselName", StringType(), True),
    StructField("IMO", StringType(), True),
    StructField("CallSign", StringType(), True),
    StructField("VesselType", ShortType(), True),
    StructField("Status", ShortType(), True),
    StructField("Length", FloatType(), True),
    StructField("Width", FloatType(), True),
    StructField("Draft", FloatType(), True),
    StructField("Cargo", StringType(), True),
    StructField("TransceiverClass", StringType(), False)
  ])

In [28]:
bucket_path = "gs://vessel-traffic-parquet-data/"
file_path = "raw_day/year=2024/month=1/AIS_2024_01_01.parquet"
spark_df = spark.read.schema(get_spark_schema()).format("parquet").load(bucket_path + file_path)
spark_df = spark_df.filter((f.length(f.col("MMSI")) == 9) & (f.abs(f.col("LAT")) <= 90) & (f.abs(f.col("LON")) <= 180))
vessel_profile_df = spark_df.select("MMSI", "VesselName", "IMO", "CallSign", "VesselType", "Length", "Width").distinct()
ais_df = spark_df.select("MMSI","BaseDateTime","LAT","LON","SOG","COG","Heading","Status","Draft","Cargo","TransceiverClass")

In [5]:
spark_df.schema

StructType([StructField('MMSI', StringType(), True), StructField('BaseDateTime', TimestampType(), True), StructField('LAT', DoubleType(), True), StructField('LON', DoubleType(), True), StructField('SOG', FloatType(), True), StructField('COG', FloatType(), True), StructField('Heading', FloatType(), True), StructField('VesselName', StringType(), True), StructField('IMO', StringType(), True), StructField('CallSign', StringType(), True), StructField('VesselType', ShortType(), True), StructField('Status', ShortType(), True), StructField('Length', FloatType(), True), StructField('Width', FloatType(), True), StructField('Draft', FloatType(), True), StructField('Cargo', StringType(), True), StructField('TransceiverClass', StringType(), True)])

In [23]:
spark_df.filter(f.isnull(f.col("MMSI"))).show()

+----+------------+---+---+---+---+-------+----------+---+--------+----------+------+------+-----+-----+-----+----------------+
|MMSI|BaseDateTime|LAT|LON|SOG|COG|Heading|VesselName|IMO|CallSign|VesselType|Status|Length|Width|Draft|Cargo|TransceiverClass|
+----+------------+---+---+---+---+-------+----------+---+--------+----------+------+------+-----+-----+-----+----------------+
+----+------------+---+---+---+---+-------+----------+---+--------+----------+------+------+-----+-----+-----+----------------+



In [5]:
spark_df.show()
# cols_with_nulls = [x for x in spark_df.columns if spark_df.filter(f.col(x).isNull()).count() > 0]
# cols_with_emptystring = [x for x in spark_df.columns if spark_df.filter(f.col(x).isNull()).count() > 0]

+---------+-------------------+--------+----------+----+-----+-------+--------------------+----------+--------+----------+------+------+-----+-----+-----+----------------+
|     MMSI|       BaseDateTime|     LAT|       LON| SOG|  COG|Heading|          VesselName|       IMO|CallSign|VesselType|Status|Length|Width|Draft|Cargo|TransceiverClass|
+---------+-------------------+--------+----------+----+-----+-------+--------------------+----------+--------+----------+------+------+-----+-----+-----+----------------+
|338075892|2024-01-01 00:00:03|43.65322| -70.25298| 0.0|358.8|  511.0|PILOT BOAT SPRING PT|      NULL| WDB8945|        90|     0|   0.0|  0.0|  0.0|   90|               A|
|367669550|2024-01-01 00:00:04|46.20031|-123.38573| 0.0|281.9|  141.0|   ALASKA CHALLENGER|IMO7938024| WDH9586|        30|    15|  30.0|  8.0|  0.0|   30|               A|
|367118980|2024-01-01 00:00:06|29.98534| -90.40674| 0.0| 30.1|  296.0|     CAPT J A MORGAN|IMO1186680| WDD2725|        31|    12| 115.0| 34.

In [ ]:
#documentation regarding "invalid/not accessable/default" values on:
#https://www.navcen.uscg.gov/ais-class-a-reports

#replace values for "invalid/not accessable/default" to Null for non-categorial field 
vessel_profile_df = vessel_profile_df.replace("IMO0000000", None, "IMO")
vessel_profile_df = vessel_profile_df.replace(0, None, ["Length", "Width"])
ais_df = ais_df.replace(511.0, None, "Heading")
ais_df = ais_df.replace(102.3, None, "SOG")
ais_df = ais_df.replace(360, None, "COG")
ais_df = ais_df.replace(0, None, "Draft")

#replace null to encoded "invalid/not accessable/default" values for categorial field
vessel_profile_df = vessel_profile_df.fillna(0, "VesselType")
ais_df = ais_df.fillna(15, "Status")
ais_df = ais_df.fillna(0, "Cargo")

In [10]:
ais_df.show()

+---------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+
|     MMSI|       BaseDateTime|     LAT|       LON| SOG|  COG|Heading|Status|Draft|Cargo|TransceiverClass|
+---------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+
|338075892|2024-01-01 00:00:03|43.65322| -70.25298| 0.0|358.8|   NULL|     0| NULL|   90|               A|
|367669550|2024-01-01 00:00:04|46.20031|-123.38573| 0.0|281.9|  141.0|    15| NULL|   30|               A|
|367118980|2024-01-01 00:00:06|29.98534| -90.40674| 0.0| 30.1|  296.0|    12|  3.0|   57|               A|
|367177840|2024-01-01 00:00:05|39.88654| -75.17649| 0.0|304.4|   NULL|    15| NULL|   52|               A|
|367305420|2024-01-01 00:00:06|18.33273| -64.95229| 0.0|332.6|   NULL|     0| NULL|   52|               A|
|338239081|2024-01-01 00:00:05|38.95731|  -76.4841| 0.1|111.3|   NULL|    15| NULL| NULL|               B|
|367507960|2024-01-01 00:00:02|33.753

In [11]:
vessel_profile_df.show()

+---------+-----------------+----------+--------+----------+------+-----+
|     MMSI|       VesselName|       IMO|CallSign|VesselType|Length|Width|
+---------+-----------------+----------+--------+----------+------+-----+
|367083580|         ROCKFISH|IMO8998954| WDC8223|        52|  24.0| 11.0|
|367338000|     HARVEY POWER|IMO9654232|    KVEY|        70|  92.0| 20.0|
|368128050|            ROYAL|IMO2855411| WDL4095|        60|  14.0|  5.0|
|368293190|ISABELLA JULIETTE|IMO1326320| WDN5797|        52|  24.0| 10.0|
|366982180|     C-TRACTOR 10|IMO8875463| WCO3210|        52|  25.0| 10.0|
|368339870|            KIAKI|      NULL| WDP2829|        37|  14.0|  4.0|
|538009654|    CLIPPER CLYDE|IMO9455911| V7A5002|        70| 177.0| 27.0|
|367385380|          VICTORY|      NULL| WDH8434|        37|  30.0|  7.0|
|368020980|       ASSATEAGUE|IMO9816892| WDJ8905|        31| 145.0| 17.0|
|367181290|             NYAD|      NULL| WDL8733|        36|  16.0|  5.0|
|477890100|  ZIM MOUNT BLANC|IMO993111

In [7]:
ais_df.filter((f.abs(f.col("LAT")) > 90) | (f.abs(f.col("LON")) > 180)).show()

+----+------------+---+---+---+---+-------+------+-----+-----+----------------+
|MMSI|BaseDateTime|LAT|LON|SOG|COG|Heading|Status|Draft|Cargo|TransceiverClass|
+----+------------+---+---+---+---+-------+------+-----+-----+----------------+
+----+------------+---+---+---+---+-------+------+-----+-----+----------------+



In [32]:
vessel_profile_df.filter(f.expr("VesselName like '@'")).show()

+----+----------+---+--------+----------+------+-----+
|MMSI|VesselName|IMO|CallSign|VesselType|Length|Width|
+----+----------+---+--------+----------+------+-----+
+----+----------+---+--------+----------+------+-----+



In [8]:
if vessel_profile_df.count() != vessel_profile_df.select("MMSI").distinct().count():
  vessel_profile_df.groupBy("MMSI").count().filter(f.expr("count > 1")).sort(f.desc("count")).show()

In [ ]:
vessel_profile_df.select(f.length(f.col("MMSI")).alias("MMSI-Length")).distinct().show()

+-----------+
|MMSI-Length|
+-----------+
|          1|
|          9|
|          4|
|          8|
|          7|
|         10|
|          3|
|          2|
|          6|
+-----------+



In [10]:
vessel_profile_df.select(f.length(f.col("MMSI")).alias("MMSI-Length")).groupBy(f.col("MMSI-Length")).count().show()

+-----------+-----+
|MMSI-Length|count|
+-----------+-----+
|          9|33580|
|          8|   28|
|          7|   17|
|          1|    3|
|          3|    1|
|          4|    1|
|         10|    3|
|          6|    1|
|          2|    1|
+-----------+-----+



In [11]:
vessel_profile_df.filter(f.length(f.col("MMSI")) != 9).show()

+----------+--------------------+------------+--------+----------+------+-----+
|      MMSI|          VesselName|         IMO|CallSign|VesselType|Length|Width|
+----------+--------------------+------------+--------+----------+------+-----+
|      4061|        BOOSTER 9000|        NULL| 0000000|        33|  NULL| NULL|
|   3660489|         NEPTUNE TLP|  IMO0745081| WQGV318|        99|  89.0| 60.0|
|1072211352|           SHAMBHALA|        NULL| WDG7537|        37|  30.0|  6.0|
|   3381234|            ZEEPAARD|        NULL| BO12345|        36|  NULL| NULL|
|       111|            BOOMVANG|  IMO0000001|   EB643|        90|  47.0| 35.0|
|   3791472|         LUCIUS SPAR|  IMO1108561|  WQXP40|        80|  NULL| NULL|
|        11|        CONSTITUTION|  IMO0000007|  GC 680|        90|  53.0| 38.0|
|   1234567|            ST000001|        NULL| BO12345|         1|  NULL| NULL|
|   1193046|           NAUTICAST|IMO303174162|  D11233|        50| 220.0| 43.0|
|   1056261|      CHAMPAGNE CHER|  IMO10

In [17]:
ais_df.filter(f.col("MMSI") == '36968098').show()

+--------+-------------------+--------+---------+---+-----+-------+------+-----+-----+----------------+
|    MMSI|       BaseDateTime|     LAT|      LON|SOG|  COG|Heading|Status|Draft|Cargo|TransceiverClass|
+--------+-------------------+--------+---------+---+-----+-------+------+-----+-----+----------------+
|36968098|2024-01-01 00:00:07| 36.9512|-76.32952|0.1|128.8|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:01:17| 36.9512|-76.32949|0.1|123.6|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:02:27|36.95116|-76.32936|0.0|126.3|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:04:48|36.95121|-76.32954|0.1|125.6|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:07:06|36.95118|-76.32943|0.1|125.4|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:03:37|36.95123|-76.32957|0.1|110.1|   NULL|     0| NULL|    0|               A|
|36968098|2024-01-01 00:08:07|36.95122|-76.32946|0.0|113.0|   NU

In [15]:
ais_df.filter(f.col("MMSI") == '99043470').show()

+--------+-------------------+--------+--------+---+-----+-------+------+-----+-----+----------------+
|    MMSI|       BaseDateTime|     LAT|     LON|SOG|  COG|Heading|Status|Draft|Cargo|TransceiverClass|
+--------+-------------------+--------+--------+---+-----+-------+------+-----+-----+----------------+
|99043470|2024-01-01 04:48:22|45.21632|-122.682|0.0|220.2|   NULL|     0| NULL| NULL|               A|
+--------+-------------------+--------+--------+---+-----+-------+------+-----+-----+----------------+



In [16]:
ais_df.filter(f.col("MMSI") == '91481544').show()

+--------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+
|    MMSI|       BaseDateTime|     LAT|       LON| SOG|  COG|Heading|Status|Draft|Cargo|TransceiverClass|
+--------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+
|91481544|2024-01-04 05:25:28|41.99946|-108.11237|25.8|314.2|  142.0|     4| NULL| NULL|               A|
+--------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+



In [6]:
ais_df.filter(f.length(f.col("MMSI")) != 9).count()

110673

In [29]:
delta_phi = f.radians(f.expr("lead_LAT - LAT"))
delta_lambda = f.radians(f.expr("lead_LON - LON"))
a = f.pow(f.sin(delta_phi / 2), 2) + f.cos(f.radians(f.col("LAT"))) * f.cos(f.radians(f.col("lead_LAT"))) * f.pow(f.sin(delta_lambda)/2,2)
c = 2 * f.atan2(f.sqrt(a), f.sqrt(1-a))
d = 6371 * c

In [ ]:
print(*[col for col in ais_df.columns if col != "SOG"])

MMSI BaseDateTime LAT LON COG Heading Status Draft Cargo TransceiverClass


In [50]:
def col_select(col:str):
  if col == "SOG":
    return f.col("SOG").alias("SOG'")
  else:
    return f.col(col)

In [51]:
windowSpec = Window.partitionBy(f.col("MMSI")).orderBy(f.asc(f.col("BaseDateTime")))
#test_df = ais_df.select([col for col in ais_df.columns if col != "SOG"])
test_df = ais_df.select(*[col_select(col) for col in ais_df.columns], 
                      f.lead(f.col("LAT")).over(windowSpec).alias("lead_LAT"),
                      f.lead(f.col("LON")).over(windowSpec).alias("lead_LON"),
                      f.lead(f.col("BaseDateTime")).over(windowSpec).alias("lead_time"),
                      d.alias("d"),
                      "SOG"
                      )

In [ ]:
test_df.filter(f.expr("SOG > 50")).show()

+---------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+--------+----------+-------------------+------------------+----+
|     MMSI|       BaseDateTime|     LAT|       LON|SOG'|  COG|Heading|Status|Draft|Cargo|TransceiverClass|lead_LAT|  lead_LON|          lead_time|                 d| SOG|
+---------+-------------------+--------+----------+----+-----+-------+------+-----+-----+----------------+--------+----------+-------------------+------------------+----+
|209470000|2024-01-01 02:47:34| 28.2219| -89.07157|14.2|144.9|  143.0|     0| 12.9|   79|               A|28.01173| -88.89327|2024-01-01 03:54:09| 29.18767699366881|14.2|
|210959000|2024-01-01 17:07:05|27.80652|   -86.833|11.6| 24.3|   25.0|     0|  5.2|   70|               A|28.24914| -86.62681|2024-01-01 21:20:04| 53.21568107121288|11.6|
|212370000|2024-01-01 02:09:54|28.30315| -86.60223|10.6|202.0|  201.0|     0|  4.4|   70|               A| 28.1672| -86.66289|2024-01-01 03:00:14

In [37]:
test_df.show()

+---------+-------------------+--------+---------+----+-----+-------+------+-----+-----+----------------+--------+---------+------------------+
|     MMSI|       BaseDateTime|     LAT|      LON| SOG|  COG|Heading|Status|Draft|Cargo|TransceiverClass|lead_LAT| lead_LON|                 d|
+---------+-------------------+--------+---------+----+-----+-------+------+-----+-----+----------------+--------+---------+------------------+
|209425000|2024-01-01 00:36:13|16.12003|-66.55203|16.2|318.9|  322.0|     0|  7.7|   71|               A|16.12443|-66.55612|0.6559387806163122|
|209425000|2024-01-01 00:37:31|16.12443|-66.55612|16.3|317.0|  322.0|     0|  7.7|   71|               A|16.12821|-66.55964|0.5639556381859766|
|209425000|2024-01-01 00:38:38|16.12821|-66.55964|16.3|317.5|  322.0|     0|  7.7|   71|               A|16.13187|-66.56307|0.5475991626106907|
|209425000|2024-01-01 00:39:43|16.13187|-66.56307|16.2|316.6|  322.0|     0|  7.7|   71|               A|16.13628|-66.56727|0.6646250904